# P1 REST API

- This Jupyter notebook is an example of how to access the RestAPI interface, described at:
  https://doc.particle.one/

# Credentials / Settings

In [1]:
%load_ext autoreload
%autoreload 2

import json
import os

import pandas as pd
import requests

In [2]:
# Enter your token here.
# You can get your token by signing up at `www.particle.one`.
# P1_API_TOKEN = "YOUR_TOKEN_HERE"
# An example token is like:
# P1_API_TOKEN = "e44e7c6b04ef3ea1cfb7a8a67db74751c177259e"
P1_API_TOKEN = os.environ["P1_API_TOKEN"]
print("P1_API_TOKEN=", P1_API_TOKEN)

HEADERS = {
    "Authorization": f"Token {P1_API_TOKEN}",
    "Content-Type": "application/json",
}

P1_API_TOKEN= e44e7c6b04ef3ea1cfb7a8a67db74751c177259e


# Search query structure

Search query is a Python `dict` with the following structure:
```python
query = {
    "text": "",
    "commodity": [],
    "business_category": "",
    "country": [],
    "frequency": []
}
```
The fields are:
- `text`: string. Works as a filter. Free text. Everything that have no match with this phrase will be filtered out.
- `commodity`: list of strings. Works as a filter. You can find valid values in paragraph 7.1 of this notebook.
- `business_category`: string. Works as a filter. You can find valid values in paragraph 7.2 of this notebook.
- `country`: list of strings. Works as a filter. You can find valid values in paragraph 7.3 of this notebook.
- `frequency`: list of strings. Works as a filter. You can find valid values in paragraph 7.4 of this notebook.

Combination of fields work with logical operator AND.
E.g. you will get all records that satisfy all filters.

`text` **AND** `commodity` **AND** `business_category` **AND** `country` **AND** `frequency`

# Imports

# POST data-api/v1/search-count/
Returns count for the given query.

In [3]:
# Build entrypoint url.
base_url = "https://data.particle.one"
count_url = os.path.join(base_url, "data-api/v1/search-count/")
print("count_url=", count_url)

count_url= https://data.particle.one/data-api/v1/search-count/


In [4]:
# Prepare query.
query = {
    "text": "",
    "commodity": ["Corn"],
    "business_category": "",
    "country": [],
    "frequency": [],
}
payload = json.dumps(query)

In [5]:
# Perform query.
response = requests.request("POST", count_url, headers=HEADERS, data=payload)
data = json.loads(response.text.encode("utf8"))
print("data=", data)

data= {'message': 'OK', 'count': 3715}


# POST data-api/v1/search/

- It returns the first chunk of the payload metadata for the given query, where a chunk is 1000 records.
- It also returns `scroll_id` to get the next portion of the data.

In [6]:
search_url = os.path.join(base_url, "data-api/v1/search/")
print("search_url=", search_url)

search_url= https://data.particle.one/data-api/v1/search/


In [7]:
# Prepare query.
query = {
    "text": "Gas",
    "commodity": [],
    "business_category": "",
    "country": [],
    "frequency": [],
}
payload = json.dumps(query)

In [8]:
# Perform query.
response = requests.request("POST", search_url, headers=HEADERS, data=payload)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

assert "detail" not in data, data

print("total_count=", data["total_count"])

# Saving scroll_id for the next query.
scroll_id = data["scroll_id"]
print("scroll_id=", scroll_id)

df = pd.DataFrame.from_records(data["rows"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'scroll_id', 'total_count', 'rows']
total_count= 452593
scroll_id= DXF1ZXJ5QW5kRmV0Y2gBAAAAAABs8aYWQUFleVN5Ym9RT0daMWhMQU1KNXJ4dw==
df.shape= (1000, 8)
df.head()=


,name,commodity,payload_id,business_category,country,frequency,unit,start_date
0,"""Non-durable goods – 04 Housing,water,electric...",Natural Gas,fd978f2dd9dde797bfb90315474e2efefd64c83d,Undefined,United Kingdom,Annual,None,1944-01-01
1,"""Non-durable goods – 04 Housing,water,electric...",Natural Gas,3799f257a7e7d03686893c9b85de0215a43272e8,Undefined,United Kingdom,Annual,None,1955-01-01
2,"""Non-durable goods – 04 Housing,water,electric...",Natural Gas,1ed8eff51a6361383318ca3c51e41cbeb05def05,Undefined,United Kingdom,Quarterly,None,1955-01-01
3,"""Non-durable goods – 04 Housing,water,electric...",Electricity,fd978f2dd9dde797bfb90315474e2efefd64c83d,Undefined,United Kingdom,Annual,None,1944-01-01
4,"""Non-durable goods – 04 Housing,water,electric...",Electricity,3799f257a7e7d03686893c9b85de0215a43272e8,Undefined,United Kingdom,Annual,None,1955-01-01


# GET data-api/v1/search-scroll/?scroll_id=

In [9]:
# Build entrypoint url.

# We use scroll id from the previous query.
search_scroll_url = os.path.join(
    base_url, f"data-api/v1/search-scroll/?scroll_id={scroll_id}"
)
print("search_scroll_url=", search_scroll_url)

search_scroll_url= https://data.particle.one/data-api/v1/search-scroll/?scroll_id=DXF1ZXJ5QW5kRmV0Y2gBAAAAAABs8aYWQUFleVN5Ym9RT0daMWhMQU1KNXJ4dw==


In [10]:
# Perform query.

response = requests.request("GET", search_scroll_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))
print("data['rows'][0]=", data["rows"][0])

df = pd.DataFrame.from_records(data["rows"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'scroll_id', 'rows']
data['rows'][0]= {'name': 'Alabama (with State Offshore) Natural Gas Liquids Lease Condensate – Proved Reserves Extensions – Million Barrels', 'commodity': 'Natural Gas', 'payload_id': 'b17aefbfef91149d965d6668b0b9004d3e4c829a', 'business_category': 'Undefined', 'country': 'United States', 'frequency': 'Annual', 'unit': 'bbl, in millions', 'start_date': None}
df.shape= (1000, 8)
df.head()=


,name,commodity,payload_id,business_category,country,frequency,unit,start_date
0,Alabama (with State Offshore) Natural Gas Liqu...,Natural Gas,b17aefbfef91149d965d6668b0b9004d3e4c829a,Undefined,United States,Annual,"bbl, in millions",None
1,Alabama (with State Offshore) Natural Gas Liqu...,Natural Gas,307d1fd5008a3c0c6055205f1f412bc02b4209a7,Undefined,United States,Annual,"bbl, in millions",None
2,Alabama (with State Offshore) Natural Gas Liqu...,Natural Gas,52522653edb88ff5c5fbcf6806a22c8f11ac86c7,Undefined,United States,Annual,"bbl, in millions",None
3,Alabama (with State Offshore) Natural Gas Liqu...,Natural Gas,8dee70358b27a7cd26977944a5dbacd2a76218dd,Undefined,United States,Annual,"bbl, in millions",None
4,Alabama (with State Offshore) Natural Gas Liqu...,Natural Gas,ca80ff1e7a888dd798ad0fe00d9d095ae0f2457c,Undefined,United States,Annual,"bbl, in millions",None


# GET data-api/v1/payload/?payload_id=
Returns payload for the given `payload_id`

In [11]:
# Build entrypoint url.

# We use one of the `payload_id` from one of the previous queries.
payload_id = "8f26ba4734df3a62352cce9d64987d64da54b400"
payload_url = os.path.join(
    base_url, f"data-api/v1/payload/?payload_id={payload_id}"
)
print("payload_url=", payload_url)

payload_url= https://data.particle.one/data-api/v1/payload/?payload_id=8f26ba4734df3a62352cce9d64987d64da54b400


In [12]:
# Perform query.
response = requests.request("GET", payload_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["payload_data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'payload_data']
df.shape= (166, 4)
df.head()=


,original_period,original_value,period,value
0,1980,0.0,1980-01-01T00:00:00,0
1,1981,0.0,1981-01-01T00:00:00,0
2,1982,0.0,1982-01-01T00:00:00,0
3,1983,0.0,1983-01-01T00:00:00,0
4,1984,0.0,1984-01-01T00:00:00,0


# Helpers

To search we use several predefined lists of names for each parameter of metadata. Such as:
- `commodity`
- `business-category`
- `country`
- `frequency`

Each parameter has its own set of valid names.

## GET data-api/v1/commodities/

In [13]:
# Build entrypoint url.
commodities_url = os.path.join(base_url, "data-api/v1/commodities/")
print("commodities_url=", commodities_url)

commodities_url= https://data.particle.one/data-api/v1/commodities/


In [14]:
# Perform query.
response = requests.request("GET", commodities_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (77, 1)
df.head()=


,name
0,Aluminum
1,Benzene
2,Biodiesel
3,Biofuel
4,Butadiene


## GET data-api/v1/business-categories/

In [15]:
# Build entrypoint url.
bc_url = os.path.join(base_url, "data-api/v1/business-categories/")
print("bc_url=", bc_url)

bc_url= https://data.particle.one/data-api/v1/business-categories/


In [16]:
# Perform query.

response = requests.request("GET", bc_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (4, 1)
df.head()=


,name
0,Downstream
1,Midstream
2,Undefined
3,Upstream


## GET data-api/v1/countries/

In [17]:
# Build entrypoint url.
countries_url = os.path.join(base_url, "data-api/v1/countries/")
print("countries_url=", countries_url)

countries_url= https://data.particle.one/data-api/v1/countries/


In [18]:
# Perform query.
response = requests.request("GET", countries_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (237, 1)
df.head()=


,name
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra


## GET data-api/v1/frequencies/

In [19]:
# Build entrypoint url.
frequencies_url = os.path.join(base_url, "data-api/v1/frequencies/")
print(frequencies_url)

https://data.particle.one/data-api/v1/frequencies/


In [20]:
# Perform query.
response = requests.request("GET", frequencies_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (11, 1)
df.head()=


,name
0,Hourly - UTC Time
1,Business daily
2,Daily
3,Weekly
4,Bi-Weekly
